**44. Wildcard Matching**

**Hard**

**Companies**: Adobe Amazon Bloomberg Coursera Cruise Automation Facebook Goldman Sachs Google Microsoft Snapchat Twitter Two Sigma

Given an input string (s) and a pattern (p), implement wildcard pattern matching with support for '?' and '\*' where:

'?' Matches any single character.
'\*' Matches any sequence of characters (including the empty sequence).
The matching should cover the entire input string (not partial).

**Example 1:**

```python
Input: s = "aa", p = "a"
Output: false
```

**Explanation:** "a" does not match the entire string "aa".

**Example 2:**

```python
Input: s = "aa", p = "*"
Output: true
```

**Explanation:** '\*' matches any sequence.

**Example 3:**

```python
Input: s = "cb", p = "?a"
Output: false
```

**Explanation:** '?' matches 'c', but the second letter is 'a', which does not match 'b'.

**Constraints:**

- 0 <= s.length, p.length <= 2000
- s contains only lowercase English letters.
- p contains only lowercase English letters, '?' or '\*'.


In [ ]:
# =====================================================
# 🧠 Approach 1: Recursive Backtracking (Brute Force)
# -----------------------------------------------------
# 🔹 Algorithm:
#   1. If both s and p are empty → True.
#   2. If pattern starts with '*':
#        - '*' can match 0 chars → recurse(s, p[1:])
#        - '*' can match 1+ chars → recurse(s[1:], p)
#   3. If first characters match or pattern starts with '?':
#        - recurse(s[1:], p[1:])
#   4. Otherwise → False.
#
# 🔹 Time Complexity: Exponential (O(2^(m+n)))
# 🔹 Space Complexity: O(m+n) recursion depth
# 🔹 Works only for small inputs (not efficient for n ≤ 2000).
# =====================================================
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        def helper(i, j):
            # Base cases
            if i == len(s) and j == len(p):
                return True
            if j == len(p):
                return False
            if i == len(s):
                # Remaining pattern should be all '*'
                return all(x == '*' for x in p[j:])

            # Matching cases
            if p[j] == s[i] or p[j] == '?':
                return helper(i + 1, j + 1)
            if p[j] == '*':
                return helper(i, j + 1) or helper(i + 1, j)
            return False

        return helper(0, 0)


In [ ]:
# =====================================================
# 🧠 Approach 2: Recursion + Memoization (Top-Down DP)
# -----------------------------------------------------
# 🔹 Idea:
#   Optimize brute force by caching overlapping subproblems.
#   Memoize the results for each (i, j) state.
#
# 🔹 Algorithm:
#   1. Define dp(i, j): whether s[i:] matches p[j:].
#   2. Store results in memo dict to avoid recomputation.
# =====================================================
from functools import lru_cache

class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        @lru_cache(None)
        def dp(i, j):
            if i == len(s) and j == len(p):
                return True
            if j == len(p):
                return False
            if i == len(s):
                return all(x == '*' for x in p[j:])

            if p[j] == s[i] or p[j] == '?':
                return dp(i + 1, j + 1)
            if p[j] == '*':
                return dp(i, j + 1) or dp(i + 1, j)
            return False

        return dp(0, 0)


In [ ]:
# =====================================================
# 🧠 Approach 3: Bottom-Up DP (Tabulation)
# -----------------------------------------------------
# 🔹 Idea:
#   dp[i][j] = True if s[:i] matches p[:j]
#
# 🔹 Transitions:
#   1️⃣ If p[j-1] == s[i-1] or p[j-1] == '?':
#           dp[i][j] = dp[i-1][j-1]
#   2️⃣ If p[j-1] == '*':
#           dp[i][j] = dp[i][j-1] or dp[i-1][j]
#           ('*' as empty or match one/more chars)
#
# 🔹 Base Cases:
#   - dp[0][0] = True
#   - dp[0][j] = True only if p[:j] are all '*'
#
# 🔹 Time Complexity: O(m × n)
# 🔹 Space Complexity: O(m × n)
# =====================================================
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        m, n = len(s), len(p)
        dp = [[False] * (n + 1) for _ in range(m + 1)]
        dp[0][0] = True

        # Empty string can match prefix of '*'s
        for j in range(1, n + 1):
            if p[j - 1] == '*':
                dp[0][j] = dp[0][j - 1]

        # Fill table
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if p[j - 1] == s[i - 1] or p[j - 1] == '?':
                    dp[i][j] = dp[i - 1][j - 1]
                elif p[j - 1] == '*':
                    dp[i][j] = dp[i][j - 1] or dp[i - 1][j]

        return dp[m][n]


In [ ]:
# =====================================================
# 🧠 Approach 4: Space Optimized DP
# -----------------------------------------------------
# 🔹 Idea:
#   Since dp[i][j] depends only on previous row and current row,
#   we can reduce the space from O(m×n) → O(n).
#
# 🔹 Time Complexity: O(m × n)
# 🔹 Space Complexity: O(n)
# =====================================================
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        m, n = len(s), len(p)
        prev = [False] * (n + 1)
        prev[0] = True

        for j in range(1, n + 1):
            if p[j - 1] == '*':
                prev[j] = prev[j - 1]

        for i in range(1, m + 1):
            curr = [False] * (n + 1)
            for j in range(1, n + 1):
                if p[j - 1] == s[i - 1] or p[j - 1] == '?':
                    curr[j] = prev[j - 1]
                elif p[j - 1] == '*':
                    curr[j] = curr[j - 1] or prev[j]
            prev = curr

        return prev[n]


In [ ]:
# =====================================================
# 🧠 Approach 5: Greedy Two-Pointer
# -----------------------------------------------------
# 🔹 Idea:
#   Keep track of '*' position and last match index in s.
#   When mismatch occurs, backtrack to the last '*'.
#
# 🔹 Algorithm:
#   1. Use two pointers i (s) and j (p).
#   2. If match or '?', move both.
#   3. If '*', record star position and move j.
#   4. If mismatch but we have a previous '*':
#         - Backtrack: move star to match one more char.
#   5. If no star available → return False.
#   6. Skip trailing '*' in pattern.
#
# 🔹 Time Complexity: O(m + n)
# 🔹 Space Complexity: O(1)
# 🔹 Fastest practical solution.
# =====================================================
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        i = j = 0
        star = -1
        match = 0
        while i < len(s):
            if j < len(p) and (p[j] == s[i] or p[j] == '?'):
                i += 1
                j += 1
            elif j < len(p) and p[j] == '*':
                star = j
                match = i
                j += 1
            elif star != -1:
                j = star + 1
                match += 1
                i = match
            else:
                return False
        while j < len(p) and p[j] == '*':
            j += 1
        return j == len(p)
